In [19]:
import pandas as pd
data=pd.read_csv("amazon_baby.csv",error_bad_lines=False)
data


,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5
...,...,...,...
183526,Baby Teething Necklace for Mom Pretty Donut Sh...,Such a great idea! very handy to have and look...,5
183527,Baby Teething Necklace for Mom Pretty Donut Sh...,This product rocks! It is a great blend of fu...,5
183528,Abstract 2 PK Baby / Toddler Training Cup (Pink),This item looks great and cool for my kids.......,5
183529,"Baby Food Freezer Tray - Bacteria Resistant, B...",I am extremely happy with this product. I have...,5


In [21]:
!pip3 install Pystemmer 

In [23]:
data_text=data[["review"]]
data_text['index']=data_text.index

documents=data_text

<ipython-input-23-d1db491ca8c1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_text['index']=data_text.index


In [25]:
documents

,review,index
0,"These flannel wipes are OK, but in my opinion ...",0
1,it came early and was not disappointed. i love...,1
2,Very soft and comfortable and warmer than it l...,2
3,This is a product well worth the purchase. I ...,3
4,All of my kids have cried non-stop when I trie...,4
...,...,...
183526,Such a great idea! very handy to have and look...,183526
183527,This product rocks! It is a great blend of fu...,183527
183528,This item looks great and cool for my kids.......,183528
183529,I am extremely happy with this product. I have...,183529


In [27]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
import nltk.stem as stemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/deth/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [29]:
stemmer=PorterStemmer()
def lemmatize_stemming(text):
      return stemmer.stem(WordNetLemmatizer().lemmatize(text,pos='v'))

def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) >3:
              result.append(lemmatize_stemming(token))
  
    return result

In [31]:
doc_sample=documents[documents['index']==4310].values[0][0]
print('original document')
words=[]
for word in doc_sample.split(' '):
      words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document
['They', 'were', 'for', 'my', 'daughter', '&', 'she', 'had', 'some', 'shells', 'like', 'this', 'before', 'and', 'liked', 'them.', 'My', 'sister', 'started', 'with', 'some', 'of', 'these', '40', 'years', 'ago', 'with', 'her', 'oldest', 'daughter', '-', 'got', 'me', 'some', 'for', 'me', 'when', 'my', 'daughter', 'was', 'born', '30', 'years', 'ago', '-', 'then', 'now', 'my', 'daughter', 'is', 'using', 'these', '-', 'we', 'are', 'great', 'fans.']


 tokenized and lemmatized document: 
['daughter', 'shell', 'like', 'like', 'sister', 'start', 'year', 'oldest', 'daughter', 'daughter', 'bear', 'year', 'daughter', 'great', 'fan']


In [33]:
documents.dropna(subset = ["review"], inplace=True) # drop those rows which have NaN value cells

<ipython-input-33-00cd468fc166>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  documents.dropna(subset = ["review"], inplace=True) # drop those rows which have NaN value cells


In [35]:
processed_docs = documents['review'].map(preprocess)
processed_docs[:10]

0    [flannel, wipe, opinion, worth, keep, order, s...
1    [come, earli, disappoint, love, planet, wise, ...
2    [soft, comfort, warmer, look, size, perfectli,...
3    [product, worth, purchas, like, posit, ingeni,...
4    [kid, cri, stop, tri, ween, pacifi, thumbuddi,...
5    [binki, fairi, come, hous, special, gift, book...
6    [love, book, bind, tightli, abl, alot, photo, ...
7    [perfect, parent, abl, track, babi, feed, slee...
8    [friend, pin, product, pinterest, decid, whirl...
9    [easi, nanni, record, event, happen, babi, hom...
Name: review, dtype: object

In [37]:
dictionary=gensim.corpora.Dictionary(processed_docs)
count=0

for k,v in dictionary.iteritems():
    print(k,v)
    count+=1
    if count>10:
        break
        

0 blue
1 boyfor
2 cloth
3 countwhich
4 face
5 flannel
6 hand
7 handl
8 higher
9 issu
10 keep


In [39]:
dictionary.filter_extremes(no_below=15,no_above=0.5,keep_n=100000)

In [41]:
bow_corpus=[dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(2, 1),
 (4, 1),
 (36, 3),
 (60, 1),
 (64, 1),
 (75, 1),
 (177, 1),
 (283, 1),
 (288, 1),
 (384, 1),
 (418, 1),
 (541, 1),
 (655, 1),
 (684, 1),
 (696, 1),
 (705, 1),
 (2417, 1),
 (2427, 1),
 (2472, 1),
 (4219, 1)]

In [43]:
bow_doc_4310=bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 2 ("face") appears 1 time.
Word 4 ("hand") appears 1 time.
Word 36 ("love") appears 3 time.
Word 60 ("littl") appears 1 time.
Word 64 ("product") appears 1 time.
Word 75 ("great") appears 1 time.
Word 177 ("older") appears 1 time.
Word 283 ("turn") appears 1 time.
Word 288 ("enjoy") appears 1 time.
Word 384 ("carri") appears 1 time.
Word 418 ("infant") appears 1 time.
Word 541 ("stuff") appears 1 time.
Word 655 ("free") appears 1 time.
Word 684 ("wear") appears 1 time.
Word 696 ("watch") appears 1 time.
Word 705 ("cook") appears 1 time.
Word 2417 ("stave") appears 1 time.
Word 2427 ("oven") appears 1 time.
Word 2472 ("dinner") appears 1 time.
Word 4219 ("cuddl") appears 1 time.


In [45]:
from gensim import corpora, models
tfidf=models.TfidfModel(bow_corpus)
corpus_tfidf=tfidf[bow_corpus]

from pprint import pprint

for doc in corpus_tfidf:
  pprint(doc)
  break

[(0, 0.15091501593165862),
 (1, 0.23393853569876197),
 (2, 0.1255369345636988),
 (3, 0.22598623908509632),
 (4, 0.09893873572438135),
 (5, 0.12114001415352099),
 (6, 0.16991794106277847),
 (7, 0.11531090952299693),
 (8, 0.10275819221163739),
 (9, 0.14398977201310792),
 (10, 0.12706961807603134),
 (11, 0.058789838547190325),
 (12, 0.07374427507753085),
 (13, 0.1975073447434405),
 (14, 0.222002426625964),
 (15, 0.1738036264342695),
 (16, 0.0987763115139214),
 (17, 0.12337834464728409),
 (18, 0.09206114474407778),
 (19, 0.12067577899359858),
 (20, 0.18354165227218894),
 (21, 0.17707136078831798),
 (22, 0.09944349619585185),
 (23, 0.2121994764612386),
 (24, 0.19151610875759867),
 (25, 0.17078868919605977),
 (26, 0.29896491821974913),
 (27, 0.5293714406926228),
 (28, 0.10374295805010908)]


In [47]:
lda_model=gensim.models.LdaMulticore(bow_corpus,num_topics=10,id2word=dictionary,passes=2,workers=2)

In [49]:
for idx, topic in lda_model.print_topics(-1):
      print('Topic:{}\nwords:{}'.format(idx,topic))
  

Topic:0
words:0.020*"easi" + 0.018*"love" + 0.017*"great" + 0.016*"like" + 0.016*"clean" + 0.015*"food" + 0.015*"diaper" + 0.015*"babi" + 0.014*"chair" + 0.012*"littl"
Topic:1
words:0.034*"babi" + 0.021*"soft" + 0.017*"wash" + 0.017*"sleep" + 0.015*"blanket" + 0.015*"like" + 0.015*"cover" + 0.012*"size" + 0.012*"mattress" + 0.012*"crib"
Topic:2
words:0.050*"babi" + 0.037*"love" + 0.022*"month" + 0.015*"play" + 0.015*"like" + 0.014*"littl" + 0.012*"time" + 0.012*"gift" + 0.011*"daughter" + 0.011*"great"
Topic:3
words:0.025*"diaper" + 0.021*"product" + 0.015*"work" + 0.012*"time" + 0.010*"review" + 0.010*"order" + 0.010*"return" + 0.009*"cloth" + 0.008*"money" + 0.008*"amazon"
Topic:4
words:0.042*"love" + 0.022*"great" + 0.020*"daughter" + 0.020*"year" + 0.019*"easi" + 0.014*"littl" + 0.013*"month" + 0.013*"buy" + 0.012*"recommend" + 0.011*"like"
Topic:5
words:0.045*"stroller" + 0.026*"monitor" + 0.021*"babi" + 0.011*"camera" + 0.011*"great" + 0.010*"work" + 0.009*"like" + 0.009*"carrier

In [51]:
lda_model_tfidf=gensim.models.LdaMulticore(corpus_tfidf,num_topics=10,id2word=dictionary,passes=2,workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
      print('Topic: {} Word: {}'.format(idx,topic))

Topic: 0 Word: 0.013*"mattress" + 0.012*"blanket" + 0.011*"sheet" + 0.009*"crib" + 0.009*"sleep" + 0.009*"soft" + 0.007*"babi" + 0.006*"swaddl" + 0.006*"wash" + 0.005*"fit"
Topic: 1 Word: 0.029*"diaper" + 0.014*"cloth" + 0.010*"wipe" + 0.008*"wash" + 0.007*"chang" + 0.007*"smell" + 0.006*"pocket" + 0.006*"absorb" + 0.005*"great" + 0.005*"insert"
Topic: 2 Word: 0.028*"bottl" + 0.021*"pump" + 0.013*"nippl" + 0.011*"pacifi" + 0.008*"milk" + 0.007*"breast" + 0.007*"work" + 0.007*"medela" + 0.007*"spoon" + 0.007*"babi"
Topic: 3 Word: 0.021*"monitor" + 0.010*"batteri" + 0.009*"sound" + 0.008*"camera" + 0.008*"work" + 0.007*"night" + 0.007*"swing" + 0.007*"light" + 0.006*"babi" + 0.006*"unit"
Topic: 4 Word: 0.027*"seat" + 0.012*"instal" + 0.010*"gate" + 0.008*"easi" + 0.006*"britax" + 0.006*"open" + 0.006*"strap" + 0.006*"door" + 0.005*"carseat" + 0.005*"work"
Topic: 5 Word: 0.016*"love" + 0.012*"play" + 0.011*"babi" + 0.010*"toy" + 0.010*"month" + 0.008*"daughter" + 0.007*"littl" + 0.006*"gr

In [53]:
processed_docs[4310]

['daughter',
 'shell',
 'like',
 'like',
 'sister',
 'start',
 'year',
 'oldest',
 'daughter',
 'daughter',
 'bear',
 'year',
 'daughter',
 'great',
 'fan']

In [55]:
for index, score in sorted(lda_model[bow_corpus[4310]],key=lambda tup:-1*tup[1]):
  print("\nScore:{}\t \nTopic:{}".format(score,lda_model.print_topic(index,10)))


Score:0.8341153860092163	 
Topic:0.050*"babi" + 0.037*"love" + 0.022*"month" + 0.015*"play" + 0.015*"like" + 0.014*"littl" + 0.012*"time" + 0.012*"gift" + 0.011*"daughter" + 0.011*"great"

Score:0.13108089566230774	 
Topic:0.025*"open" + 0.015*"work" + 0.015*"gate" + 0.011*"easi" + 0.010*"smell" + 0.010*"door" + 0.009*"close" + 0.009*"wall" + 0.009*"like" + 0.009*"need"


In [57]:
from gensim.test.utils import common_dictionary, common_corpus
from gensim.models import LsiModel

lsi_model=LsiModel(bow_corpus,num_topics=10,id2word=dictionary)

In [59]:
for idx, topic in lsi_model.print_topics(-1):
      print('Topic:{}\nwords:{}'.format(idx,topic))

Topic:0
words:0.377*"babi" + 0.347*"seat" + 0.243*"like" + 0.205*"love" + 0.199*"stroller" + 0.164*"great" + 0.161*"time" + 0.149*"month" + 0.138*"easi" + 0.137*"littl"
Topic:1
words:-0.801*"seat" + 0.328*"babi" + -0.196*"stroller" + 0.150*"bottl" + 0.115*"diaper" + -0.084*"strap" + 0.083*"love" + 0.083*"pump" + 0.082*"work" + 0.080*"monitor"
Topic:2
words:0.688*"stroller" + -0.515*"babi" + -0.303*"seat" + 0.163*"diaper" + 0.104*"fold" + 0.086*"wheel" + 0.075*"great" + -0.066*"chair" + 0.064*"handl" + -0.061*"carrier"
Topic:3
words:-0.529*"babi" + -0.450*"stroller" + 0.421*"bottl" + 0.328*"pump" + 0.154*"work" + 0.116*"nippl" + 0.114*"seat" + 0.111*"milk" + 0.098*"time" + 0.093*"clean"
Topic:4
words:-0.547*"bottl" + 0.347*"diaper" + 0.333*"love" + -0.312*"stroller" + -0.304*"babi" + -0.257*"pump" + -0.159*"nippl" + -0.107*"milk" + 0.100*"littl" + 0.092*"great"
Topic:5
words:-0.736*"diaper" + 0.372*"love" + -0.172*"cloth" + -0.164*"babi" + 0.140*"monitor" + -0.128*"bottl" + 0.102*"great

In [61]:
lsi_model_tfidf=LsiModel(corpus_tfidf,num_topics=10,id2word=dictionary)
for idx, topic in lsi_model_tfidf.print_topics(-1):
      print('Topic: {} Word: {}'.format(idx,topic))

Topic: 0 Word: 0.197*"seat" + 0.190*"love" + 0.188*"babi" + 0.166*"great" + 0.146*"easi" + 0.143*"like" + 0.129*"month" + 0.129*"stroller" + 0.127*"product" + 0.123*"littl"
Topic: 1 Word: -0.645*"seat" + 0.400*"bottl" + -0.285*"stroller" + 0.175*"pump" + 0.136*"nippl" + -0.103*"chair" + 0.102*"diaper" + -0.098*"instal" + -0.095*"strap" + 0.089*"leak"
Topic: 2 Word: -0.597*"bottl" + -0.292*"seat" + -0.268*"pump" + -0.195*"nippl" + 0.159*"crib" + -0.135*"stroller" + 0.128*"monitor" + 0.126*"love" + 0.122*"sleep" + 0.114*"soft"
Topic: 3 Word: -0.531*"stroller" + -0.485*"diaper" + 0.268*"seat" + -0.169*"cloth" + 0.165*"monitor" + 0.138*"swing" + -0.102*"fold" + -0.095*"chang" + 0.095*"sleep" + 0.094*"play"
Topic: 4 Word: 0.583*"stroller" + -0.453*"diaper" + -0.212*"seat" + -0.207*"chair" + -0.176*"cloth" + -0.158*"potti" + -0.118*"wash" + -0.116*"clean" + 0.107*"swing" + -0.107*"chang"
Topic: 5 Word: -0.740*"gate" + -0.207*"open" + 0.170*"bottl" + -0.148*"instal" + 0.146*"seat" + 0.144*"lo

In [63]:
processed_docs[4310]

['daughter',
 'shell',
 'like',
 'like',
 'sister',
 'start',
 'year',
 'oldest',
 'daughter',
 'daughter',
 'bear',
 'year',
 'daughter',
 'great',
 'fan']

In [65]:
for index, score in sorted(lsi_model[bow_corpus[4310]],key=lambda tup:-1*tup[1]):
  print("\nScore:{}\t \nTopic:{}".format(score,lsi_model.print_topic(index,10)))


Score:1.8143148663294555	 
Topic:0.648*"love" + 0.360*"bottl" + -0.321*"monitor" + -0.203*"work" + -0.155*"pump" + -0.114*"camera" + -0.114*"product" + -0.107*"crib" + -0.104*"like" + 0.096*"nippl"

Score:1.3612839389968816	 
Topic:-0.736*"diaper" + 0.372*"love" + -0.172*"cloth" + -0.164*"babi" + 0.140*"monitor" + -0.128*"bottl" + 0.102*"great" + -0.095*"cover" + -0.090*"seat" + 0.087*"daughter"

Score:1.357350846479559	 
Topic:0.377*"babi" + 0.347*"seat" + 0.243*"like" + 0.205*"love" + 0.199*"stroller" + 0.164*"great" + 0.161*"time" + 0.149*"month" + 0.138*"easi" + 0.137*"littl"

Score:1.2222560569850827	 
Topic:-0.547*"bottl" + 0.347*"diaper" + 0.333*"love" + -0.312*"stroller" + -0.304*"babi" + -0.257*"pump" + -0.159*"nippl" + -0.107*"milk" + 0.100*"littl" + 0.092*"great"

Score:0.44860757645832894	 
Topic:-0.801*"seat" + 0.328*"babi" + -0.196*"stroller" + 0.150*"bottl" + 0.115*"diaper" + -0.084*"strap" + 0.083*"love" + 0.083*"pump" + 0.082*"work" + 0.080*"monitor"

Score:0.35727699

In [67]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.7748525738716125	 Topic: 0.025*"diaper" + 0.021*"product" + 0.015*"work" + 0.012*"time" + 0.010*"review"
Score: 0.02501954697072506	 Topic: 0.094*"seat" + 0.018*"strap" + 0.013*"instal" + 0.011*"like" + 0.011*"easi"
Score: 0.02501855418086052	 Topic: 0.032*"look" + 0.031*"color" + 0.028*"love" + 0.024*"great" + 0.022*"qualiti"
Score: 0.025016961619257927	 Topic: 0.047*"bottl" + 0.024*"pump" + 0.014*"like" + 0.014*"work" + 0.013*"nippl"
Score: 0.025016695261001587	 Topic: 0.045*"stroller" + 0.026*"monitor" + 0.021*"babi" + 0.011*"camera" + 0.011*"great"
Score: 0.025015519931912422	 Topic: 0.025*"open" + 0.015*"work" + 0.015*"gate" + 0.011*"easi" + 0.010*"smell"
Score: 0.025015370920300484	 Topic: 0.020*"easi" + 0.018*"love" + 0.017*"great" + 0.016*"like" + 0.016*"clean"
Score: 0.02501516416668892	 Topic: 0.034*"babi" + 0.021*"soft" + 0.017*"wash" + 0.017*"sleep" + 0.015*"blanket"
Score: 0.02501486800611019	 Topic: 0.042*"love" + 0.022*"great" + 0.020*"daughter" + 0.020*"year" +

In [69]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lsi_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.019396821168481973	 Topic: 0.020*"easi" + 0.018*"love" + 0.017*"great" + 0.016*"like" + 0.016*"clean"
Score: 0.007938200150906394	 Topic: 0.032*"look" + 0.031*"color" + 0.028*"love" + 0.024*"great" + 0.022*"qualiti"
Score: 0.007539470341212645	 Topic: 0.050*"babi" + 0.037*"love" + 0.022*"month" + 0.015*"play" + 0.015*"like"
Score: 0.005707382134600589	 Topic: 0.025*"diaper" + 0.021*"product" + 0.015*"work" + 0.012*"time" + 0.010*"review"
Score: 0.004477394925486246	 Topic: 0.047*"bottl" + 0.024*"pump" + 0.014*"like" + 0.014*"work" + 0.013*"nippl"
Score: 0.0009141136161267432	 Topic: 0.094*"seat" + 0.018*"strap" + 0.013*"instal" + 0.011*"like" + 0.011*"easi"
Score: 0.00026212046505711783	 Topic: 0.034*"babi" + 0.021*"soft" + 0.017*"wash" + 0.017*"sleep" + 0.015*"blanket"
Score: -0.0001826484776736863	 Topic: 0.042*"love" + 0.022*"great" + 0.020*"daughter" + 0.020*"year" + 0.019*"easi"
Score: -0.0007125234863536439	 Topic: 0.045*"stroller" + 0.026*"monitor" + 0.021*"babi" + 0.01